In [1]:
!ls ../detect4/train/weights

best.onnx  best_openvino_model	best.pt  last.pt


In [2]:
!cp -r ../detect4/train/weights/best.onnx best.onnx
!cp -r ../detect4/train/weights/best.pt best.pt

In [3]:
# yolo task=detect mode=export model=best.pt format=onnx opset=13 # write  in conslole 

In [4]:
!ls ../annotations_s/images/val | head

94a6ce7e-23f0-11e9-a7f5-e30a5f40fd48.jpg
94a73580-23f0-11e9-a924-3b95c1401421.jpg
94a75df8-23f0-11e9-a99c-b70589e9b3f5.jpg
94a7af88-23f0-11e9-aa81-5b7b64aebd80.jpg
94a8280a-23f0-11e9-abca-a32c05d084e8.jpg
94a8f348-23f0-11e9-adfa-939d45dcfc64.jpg
94a951a8-23f0-11e9-aed9-475a4d94da1c.jpg
94aa4810-23f0-11e9-b112-23992b646aea.jpg
94aa6e3a-23f0-11e9-b166-03251cbcec9b.jpg
94aa91bc-23f0-11e9-b1bf-7f297b452169.jpg
ls: write error: Broken pipe


In [5]:
from deepsparse import Pipeline

# Specify the path to your YOLO11 ONNX model
model_path = "best.onnx"

# Set up the DeepSparse Pipeline
yolo_pipeline = Pipeline.create(task="yolov8", model_path=model_path)

# Run the model on your images
images = ["../annotations_s/images/val/94a6ce7e-23f0-11e9-a7f5-e30a5f40fd48.jpg"]
pipeline_outputs = yolo_pipeline(images=images)

2024-12-15 07:53:14 deepsparse.pipeline WARNING  Could not create v2 'yolov8' pipeline, trying legacy
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.8.0 COMMUNITY | (e3778e93) (release) (optimized) (system=avx2, binary=avx2)


In [6]:
pipeline_outputs

YOLOOutput(boxes=[[[330.73236083984375, 298.5351333618164, 473.545654296875, 450.23229217529297], [24.9356689453125, 413.8678894042969, 164.16455078125, 520.8174591064453], [281.3670349121094, 21.91717529296875, 389.4460144042969, 149.87598609924316], [427.2222900390625, 24.362783432006836, 535.7445068359375, 236.3781280517578], [466.8013916015625, 452.6953582763672, 624.87451171875, 503.9292755126953], [90.71358489990234, 495.9767532348633, 201.91531372070312, 633.8363571166992]]], scores=[[0.8691822290420532, 0.7449894547462463, 0.7214380502700806, 0.7087214589118958, 0.6561912894248962, 0.635062038898468]], labels=[[15.0, 15.0, 15.0, 15.0, 26.0, 26.0]], intermediate_outputs=None)

In [7]:
from datetime import timedelta
import time
from PIL import Image
import torch
path_img  = "../annotations_s/images/val/94a6ce7e-23f0-11e9-a7f5-e30a5f40fd48.jpg"
def test_pt(model_test , path=path_img,it=20):
    torch.cpu.synchronize()
    img = Image.open(path_img)
    #warm up 
    for _ in range(5):
        model_test.predict(img,verbose=False)
        torch.cpu.synchronize()

        
    starttime = time.perf_counter()
    for _ in range(it):
        model_test.predict(img,verbose=False)
        torch.cpu.synchronize()

    return timedelta(seconds=time.perf_counter()-starttime)/it
import numpy as np  
def test_deepsparse(model_test , path=path_img,it=20):
    #warm up 
    img = np.array(Image.open(path_img))

    for _ in range(5):
        model_test(images=[img],verbose=False)
        torch.cpu.synchronize()

        
    starttime = time.perf_counter()
    for _ in range(it):
        model_test(images=[img],verbose=False)

    return timedelta(seconds=time.perf_counter()-starttime)/it
    
def test_yolo(model_test , path=path_img,it=20):
    #warm up 
    img = Image.open(path_img)

    for _ in range(5):
        model_test.predict(img,verbose=False)
        torch.cpu.synchronize()

        
    starttime = time.perf_counter()
    for _ in range(it):
        model_test.predict(img,verbose=False)

    return timedelta(seconds=time.perf_counter()-starttime)/it
    

In [1]:
from ultralytics import YOLO

model = YOLO("best.pt")  

/home/void/allinfo_new/venv3.11/venv/lib/python3.11/site-packages/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_loca

In [2]:
model.export(format='onnx')

Ultralytics YOLOv8.0.124 🚀 Python-3.11.10 torch-2.5.1+cpu CPU
Model summary (fused): 268 layers, 68156310 parameters, 0 gradients

PyTorch: starting from best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (130.4 MB)

ONNX: starting export with onnx 1.14.1 opset 19...
ONNX: export success ✅ 2.2s, saved as best.onnx (260.2 MB)

Export complete (4.5s)
Results saved to /home/void/ml_oleg/lab3
Predict:         yolo predict task=detect model=best.onnx imgsz=640 
Validate:        yolo val task=detect model=best.onnx imgsz=640 data=test.yaml 
Visualize:       https://netron.app


'best.onnx'

In [3]:
!mv best.onnx best_opset19.onnx

In [8]:
from ultralytics import YOLO

model = YOLO("best.pt")  
test_pt(model)

/home/void/allinfo_new/venv3.11/venv/lib/python3.11/site-packages/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_loca

datetime.timedelta(microseconds=390730)

In [9]:
time_deepsparse = test_deepsparse(yolo_pipeline)

In [10]:
table_str=''' 	model 	metrics/precision(B) 	metrics/recall(B) 	metrics/mAP50(B) 	metrics/mAP50-95(B) 	time 	norm_time 	metrics/precision(B)_norm 	metrics/recall(B)_norm 	metrics/mAP50(B)_norm 	metrics/mAP50-95(B)_norm
0 	torch 	0.619810 	0.436514 	0.506129 	0.353711 	652432 	0.819392 	1.033923 	1.233787 	1.291715 	1.317634
1 	dq_torch 	0.599474 	0.403165 	0.476198 	0.332022 	796239 	1.000000 	1.000000 	1.139528 	1.215327 	1.236839
2 	sq_onnx 	0.646825 	0.353800 	0.391827 	0.268444 	324608 	0.407677 	1.078988 	1.000000 	1.000000 	1.000000
3 	dq_onnx 	0.719761 	0.359985 	0.480005 	0.333922 	727462 	0.913623 	1.200654 	1.017482 	1.225043 	1.243917
'''

In [11]:
columns = table_str.split()[:11]

In [12]:
rows = table_str.split()[11:]
print(len(rows)/12)

4.0


In [13]:
rows_data = []
for i in range(4):
    rows_data.append(rows[i*12+1:(i+1)*12])


In [14]:
from collections import defaultdict
conwert_data = defaultdict(list)
for items in rows_data:
    for name,item in zip(columns,items):
        conwert_data[name].append(item)
import pandas as pd


In [15]:
dict(conwert_data)

{'model': ['torch', 'dq_torch', 'sq_onnx', 'dq_onnx'],
 'metrics/precision(B)': ['0.619810', '0.599474', '0.646825', '0.719761'],
 'metrics/recall(B)': ['0.436514', '0.403165', '0.353800', '0.359985'],
 'metrics/mAP50(B)': ['0.506129', '0.476198', '0.391827', '0.480005'],
 'metrics/mAP50-95(B)': ['0.353711', '0.332022', '0.268444', '0.333922'],
 'time': ['652432', '796239', '324608', '727462'],
 'norm_time': ['0.819392', '1.000000', '0.407677', '0.913623'],
 'metrics/precision(B)_norm': ['1.033923', '1.000000', '1.078988', '1.200654'],
 'metrics/recall(B)_norm': ['1.233787', '1.139528', '1.000000', '1.017482'],
 'metrics/mAP50(B)_norm': ['1.291715', '1.215327', '1.000000', '1.225043'],
 'metrics/mAP50-95(B)_norm': ['1.317634', '1.236839', '1.000000', '1.243917']}

In [16]:
df = pd.DataFrame(conwert_data)
df = df.drop(['norm_time','metrics/precision(B)_norm','metrics/recall(B)_norm','metrics/mAP50(B)_norm','metrics/mAP50-95(B)_norm'],axis=1)
df

,model,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),time
0,torch,0.619810,0.436514,0.506129,0.353711,652432
1,dq_torch,0.599474,0.403165,0.476198,0.332022,796239
2,sq_onnx,0.646825,0.353800,0.391827,0.268444,324608
3,dq_onnx,0.719761,0.359985,0.480005,0.333922,727462


In [17]:
model.export(format = 'openvino')

Ultralytics YOLOv8.0.124 🚀 Python-3.11.10 torch-2.5.1+cpu CPU

PyTorch: starting from best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (130.4 MB)

ONNX: starting export with onnx 1.14.1 opset 19...
ONNX: export success ✅ 2.5s, saved as best.onnx (260.2 MB)

OpenVINO: starting export with openvino 2024.5.0-17288-7975fa5da0c-refs/pull/3856/head...


[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html


OpenVINO: export success ✅ 2.0s, saved as best_openvino_model/ (260.4 MB)

Export complete (6.2s)
Results saved to /home/void/ml_oleg/lab3
Predict:         yolo predict task=detect model=best_openvino_model imgsz=640 
Validate:        yolo val task=detect model=best_openvino_model imgsz=640 data=test.yaml 
Visualize:       https://netron.app


'best_openvino_model'

In [18]:
openvino_model = YOLO('best_openvino_model')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


In [19]:
metrics_openvino = openvino_model.val(data='../test.yaml',batch=8,verbose=False)

Ultralytics YOLOv8.0.124 🚀 Python-3.11.10 torch-2.5.1+cpu CPU
Loading best_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models

val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc3d886c-5952-11ec-bcdb-d7519ad21bc8.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc3ea224-5952-11ec-bf0a-2f275af30bd6.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc3ea814-5952-11ec-bf18-4f1e8f881a41.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc3ede2e-5952-11ec-bf9d-a3e63cb46d1c.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc426d32-5952-11ec-8b26-efc80d05c8c1.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/annotations_s/images/val/fc427476-5952-11ec-8b31-f331aa9edd11.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /home/void/ml_oleg/anno

In [20]:
metrics_openvino.results_dict

{'metrics/precision(B)': 0.6066044770617985,
 'metrics/recall(B)': 0.43096028971416317,
 'metrics/mAP50(B)': 0.48948787092653767,
 'metrics/mAP50-95(B)': 0.33889499193694395,
 'fitness': 0.35395427983590333}

In [21]:
time_deepsparse

datetime.timedelta(microseconds=374442)

In [22]:
time_openvino = test_yolo(openvino_model)
time_openvino

Loading best_openvino_model for OpenVINO inference...


datetime.timedelta(microseconds=536615)

In [23]:
metrics_openvino.results_dict

{'metrics/precision(B)': 0.6066044770617985,
 'metrics/recall(B)': 0.43096028971416317,
 'metrics/mAP50(B)': 0.48948787092653767,
 'metrics/mAP50-95(B)': 0.33889499193694395,
 'fitness': 0.35395427983590333}